## Primer Cuatrimestre de 2019. Examen parcial, primera oportunidad

In [11]:
df1 = spark.read.csv("bases.csv",header=False,sep=";");
print(df1.collect())

[Row(_c0=u'1', _c1=u'NOMBRE_1', _c2=u'CABA', _c3=u'CABA', _c4=u'0,5', _c5=u'0,5'), Row(_c0=u'2', _c1=u'NOMBRE_2', _c2=u'CABA', _c3=u'CABA', _c4=u'0,5', _c5=u'0,5'), Row(_c0=u'3', _c1=u'NOMBRE_3', _c2=u'CABA', _c3=u'CABA', _c4=u'0,5', _c5=u'0,5'), Row(_c0=u'4', _c1=u'NOMBRE_4', _c2=u'BUENOSAIRES', _c3=u'LAPLATA', _c4=u'0,5', _c5=u'0,5'), Row(_c0=u'5', _c1=u'NOMBRE_5', _c2=u'SANTAFE', _c3=u'ROSARIO', _c4=u'0,5', _c5=u'0,5')]


In [22]:
df2 = spark.read.csv("mediciones.csv",header=False,sep=";");
print(df2.collect())

[Row(_c0=u'10/10/2019', _c1=u'1', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'11/10/2019', _c1=u'1', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'10/10/2019', _c1=u'1', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'10/10/2018', _c1=u'4', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'11/10/2018', _c1=u'4', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'12/10/2018', _c1=u'4', _c2=u'30', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'10/11/2018', _c1=u'4', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'11/11/2018', _c1=u'4', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'12/11/2018', _c1=u'4', _c2=u'15', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'10/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'11/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'), Row(_c0=u'12/12/2018

Posible solución.

a. Map por ID_BASE en ambos RDD.

b.Filter en RDD bases por PCIA = Buenos Aires.

c.Filter en RDD mediciones por TIMESTAMP entre 01-12-2017 y 31-12-2018.

d.Inner join entre ambos RDD.

e.MAP por ID_BASE, YEAR(TIMESTAMP), MONTH(TIMESTAMP).

f.ReduceByKey para calcular suma y contador.

g.MAP para cambiar la clave a ID_BASE y aprovechar para hacer el calculo del promedio. En los values debe
quedar el promedio, el nombre de la base y el mes/año.

h.GroupByKey

i.Filter para comparar que la tasa de un mes a otro haya variado en 50%.

j.Map para quedarse con ID_BASE y nombre.

Map innecesarios, descuento de 3 puntos. Filter tardíos

In [88]:
bases = df1.rdd
mediciones = df2.rdd

In [89]:
bases.collect()

[Row(_c0=u'1', _c1=u'NOMBRE_1', _c2=u'CABA', _c3=u'CABA', _c4=u'0,5', _c5=u'0,5'),
 Row(_c0=u'2', _c1=u'NOMBRE_2', _c2=u'CABA', _c3=u'CABA', _c4=u'0,5', _c5=u'0,5'),
 Row(_c0=u'3', _c1=u'NOMBRE_3', _c2=u'CABA', _c3=u'CABA', _c4=u'0,5', _c5=u'0,5'),
 Row(_c0=u'4', _c1=u'NOMBRE_4', _c2=u'BUENOSAIRES', _c3=u'LAPLATA', _c4=u'0,5', _c5=u'0,5'),
 Row(_c0=u'5', _c1=u'NOMBRE_5', _c2=u'SANTAFE', _c3=u'ROSARIO', _c4=u'0,5', _c5=u'0,5')]

In [90]:
mediciones.collect()

[Row(_c0=u'10/10/2019', _c1=u'1', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'11/10/2019', _c1=u'1', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'10/10/2019', _c1=u'1', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'10/10/2018', _c1=u'4', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'11/10/2018', _c1=u'4', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'12/10/2018', _c1=u'4', _c2=u'30', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'10/11/2018', _c1=u'4', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'11/11/2018', _c1=u'4', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'12/11/2018', _c1=u'4', _c2=u'15', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'10/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'11/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u

In [91]:
bases = bases.filter(lambda x: x[2] == 'BUENOSAIRES')

In [92]:
bases.collect()

[Row(_c0=u'4', _c1=u'NOMBRE_4', _c2=u'BUENOSAIRES', _c3=u'LAPLATA', _c4=u'0,5', _c5=u'0,5')]

In [93]:
mediciones = mediciones.filter(lambda x: x[0].endswith('2018'))

In [94]:
mediciones.collect()

[Row(_c0=u'10/10/2018', _c1=u'4', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'11/10/2018', _c1=u'4', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'12/10/2018', _c1=u'4', _c2=u'30', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'10/11/2018', _c1=u'4', _c2=u'25', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'11/11/2018', _c1=u'4', _c2=u'20', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'12/11/2018', _c1=u'4', _c2=u'15', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'10/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'11/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'12/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35'),
 Row(_c0=u'13/12/2018', _c1=u'4', _c2=u'10', _c3=u'30', _c4=u'35', _c5=u'35', _c6=u'35')]

In [95]:
bases = bases.map(lambda x: (x[0],(x[1],x[3])))

In [96]:
bases.collect()

[(u'4', (u'NOMBRE_4', u'LAPLATA'))]

In [97]:
mediciones = mediciones.map(lambda x: (x[1],(x[2],x[0][3:5],x[0][6:10])))

In [98]:
mediciones.collect()

[(u'4', (u'25', u'10', u'2018')),
 (u'4', (u'20', u'10', u'2018')),
 (u'4', (u'30', u'10', u'2018')),
 (u'4', (u'25', u'11', u'2018')),
 (u'4', (u'20', u'11', u'2018')),
 (u'4', (u'15', u'11', u'2018')),
 (u'4', (u'10', u'12', u'2018')),
 (u'4', (u'10', u'12', u'2018')),
 (u'4', (u'10', u'12', u'2018')),
 (u'4', (u'10', u'12', u'2018'))]

In [99]:
joins = bases.leftOuterJoin(mediciones)

In [100]:
joins.collect()

[(u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'25', u'10', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'20', u'10', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'30', u'10', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'25', u'11', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'20', u'11', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'15', u'11', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'10', u'12', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'10', u'12', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'10', u'12', u'2018'))),
 (u'4', ((u'NOMBRE_4', u'LAPLATA'), (u'10', u'12', u'2018')))]

In [101]:
joins = joins.map(lambda x: ((x[0],x[1][1][1],x[1][1][2]),(x[1][1][0],1,x[1][0][0])))

In [102]:
joins.collect()

[((u'4', u'10', u'2018'), (u'25', 1, u'NOMBRE_4')),
 ((u'4', u'10', u'2018'), (u'20', 1, u'NOMBRE_4')),
 ((u'4', u'10', u'2018'), (u'30', 1, u'NOMBRE_4')),
 ((u'4', u'11', u'2018'), (u'25', 1, u'NOMBRE_4')),
 ((u'4', u'11', u'2018'), (u'20', 1, u'NOMBRE_4')),
 ((u'4', u'11', u'2018'), (u'15', 1, u'NOMBRE_4')),
 ((u'4', u'12', u'2018'), (u'10', 1, u'NOMBRE_4')),
 ((u'4', u'12', u'2018'), (u'10', 1, u'NOMBRE_4')),
 ((u'4', u'12', u'2018'), (u'10', 1, u'NOMBRE_4')),
 ((u'4', u'12', u'2018'), (u'10', 1, u'NOMBRE_4'))]

In [103]:
joins = joins.reduceByKey(lambda x, y: (int(x[0])+int(y[0]),x[1]+y[1],x[2]))

In [104]:
joins.collect()

[((u'4', u'10', u'2018'), (75, 3, u'NOMBRE_4')),
 ((u'4', u'12', u'2018'), (40, 4, u'NOMBRE_4')),
 ((u'4', u'11', u'2018'), (60, 3, u'NOMBRE_4'))]

In [105]:
joins = joins.map(lambda x: (x[0][0],(x[1][0]/x[1][1], x[1][2], x[0][1],x[0][2])))

In [106]:
joins.collect()

[(u'4', (25, u'NOMBRE_4', u'10', u'2018')),
 (u'4', (10, u'NOMBRE_4', u'12', u'2018')),
 (u'4', (20, u'NOMBRE_4', u'11', u'2018'))]

In [108]:
group = joins.groupByKey()

In [111]:
group.collect()

[(u'4', <pyspark.resultiterable.ResultIterable at 0x7f7e917c2390>)]

In [117]:
def filter_10p(x):
    data = list(x[1])
    print(data)
    if int(data[1][2]) == 12:
        return (abs(int(data[1][0]) - int(data[2][0])) >= 0.5*int(data[2][0]))
    else:
        return False

In [130]:
resultado = group.filter(filter_10p).map(lambda x: x[0], list(x[1])[0][1])

In [131]:
resultado.collect()

[u'NOMBRE_4']